In [ ]:
from langchain import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
import os
import requests
import json
import pandas as pd
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document

# Класс-обертка для интеграции GigaChat
class GigaChatAPIWrapper:
    def __init__(self, api_key, api_url):
        self.api_key = api_key
        self.api_url = api_url

    def _make_request(self, prompt):
        headers = {
            'Authorization': f'Bearer {self.api_key}',
            'Content-Type': 'application/json'
        }
        data = {
            'prompt': prompt,
            'max_tokens': 150,
        }
        response = requests.post(self.api_url, headers=headers, json=data)
        response.raise_for_status()
        return response.json()

    def generate_response(self, prompt):
        response_data = self._make_request(prompt)
        return response_data.get('choices', [{}])[0].get('text', '').strip()

# Инициализация GigaChat API Wrapper
api_key = "ATTN_GIGACHAT_API_KEY"
api_url = "https://api.gigachat.example.com/v1/completions"
gigachat_wrapper = GigaChatAPIWrapper(api_key, api_url)

# Определение класса для использования GigaChat в LangChain
class GigaChatLLM:
    def __init__(self, api_wrapper):
        self.api_wrapper = api_wrapper

    def __call__(self, prompt):
        return self.api_wrapper.generate_response(prompt)

gigachat_llm = GigaChatLLM(gigachat_wrapper)

# Загрузка документов из файлов
collective_agreement_file = "/mnt/data/Коллективный договор.pdf"
answers_file = "/mnt/data/База знаний.xlsx"

# Извлечение текста из PDF-файла
from PyPDF2 import PdfReader
pdf_reader = PdfReader(collective_agreement_file)
collective_agreement_text = "\n".join([page.extract_text() for page in pdf_reader.pages])

# Загрузка данных из Excel-файла
answers_df = pd.read_excel(answers_file)
answers_text = "\n".join(answers_df['Ответ'].astype(str).tolist())

# Разделение текста на документы
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents([Document(page_content=collective_agreement_text), Document(page_content=answers_text)])

# Создание векторного хранилища для документов
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(documents, embeddings)

# Создание цепочки для извлечения ответов из базы данных
retrieval_chain = ConversationalRetrievalChain(
    llm=gigachat_llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)

# Пример использования чат-бота
chat_history = []

# Пример запроса к чат-боту
example_query = "Какие льготы предусмотрены для неработающих пенсионеров в коллективном договоре?"
response = retrieval_chain.run(input=example_query, chat_history=chat_history)
chat_history.append((example_query, response))
print(f"Вы: {example_query}")
print(f"Бот: {response}")

# Начало диалога с пользователем
while True:
    user_input = input("Вы: ")
    if user_input.lower() in ["exit", "quit", "выход"]:
        break

    response = retrieval_chain.run(input=user_input, chat_history=chat_history)
    chat_history.append((user_input, response))
    print(f"Бот: {response}")